# Neptune Analytics with Snowflake Data via Athena Federated Query

This notebook demonstrates how to connect PaySim data stored in Snowflake to Neptune Analytics using Athena Federated Query. We will:
1. Upload PaySim data to Snowflake and create a table
2. Set up AWS Secrets Manager for Snowflake credentials
3. Deploy Athena Snowflake connector Lambda function
4. Configure Athena data source for Snowflake
5. Import data from Snowflake into Neptune Analytics
6. Run Louvain algorithm for fraud detection
7. Export enriched graph data back to S3

## Setup

Import necessary libraries and configure logging.

In [ ]:
# Check the Python version:
import sys
assert sys.version_info >= (3, 11), "Python 3.11 or higher is required"

import logging
import os

import kagglehub
import boto3
from pathlib import Path

import dotenv

dotenv.load_dotenv()

from nx_neptune.session_manager import SessionManager

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout
)
for logger_name in [
    'nx_neptune.instance_management',
    'nx_neptune.session_manager',
]:
    logging.getLogger(logger_name).setLevel(logging.INFO)

logger = logging.getLogger(__name__)

## Configuration

Set up environment variables for Snowflake connection and AWS resources.

In [ ]:
def check_env_vars(var_names):
    values = {}
    for var_name in var_names:
        value = os.getenv(var_name)
        if not value:
            print(f"Warning: Environment Variable {var_name} is not defined")
            print(f"You can set it using: %env {var_name}=your-value")
        else:
            print(f"Using {var_name}: {value}")
        values[var_name] = value
    return values
    
env_vars = check_env_vars([
    'NETWORKX_S3_IMPORT_BUCKET_PATH',
    'NETWORKX_S3_EXPORT_BUCKET_PATH',
    'SNOWFLAKE_ACCOUNT',
    'SNOWFLAKE_USER',
    'SNOWFLAKE_PASSWORD',
    'SNOWFLAKE_DATABASE',
    'SNOWFLAKE_SCHEMA',
    'SNOWFLAKE_WAREHOUSE',
    'SNOWFLAKE_SECRET_ARN',
    'ATHENA_SPILL_BUCKET',
    'ATHENA_CATALOG_NAME',
])

s3_location_import = os.getenv('NETWORKX_S3_IMPORT_BUCKET_PATH')
s3_location_export = os.getenv('NETWORKX_S3_EXPORT_BUCKET_PATH')
snowflake_account = os.getenv('SNOWFLAKE_ACCOUNT')
snowflake_user = os.getenv('SNOWFLAKE_USER')
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')
snowflake_database = os.getenv('SNOWFLAKE_DATABASE', 'FRAUD_DETECTION')
snowflake_schema = os.getenv('SNOWFLAKE_SCHEMA', 'PUBLIC')
snowflake_warehouse = os.getenv('SNOWFLAKE_WAREHOUSE', 'COMPUTE_WH')
snowflake_secret_arn = os.getenv('SNOWFLAKE_SECRET_ARN')
athena_spill_bucket = os.getenv('ATHENA_SPILL_BUCKET')
athena_catalog_name = os.getenv('ATHENA_CATALOG_NAME', 'snowflake_catalog')
session_name = "nx-snowflake-demo"

## Step 1: Upload PaySim Data to Snowflake

Download PaySim dataset and upload to Snowflake. You'll need the Snowflake Python connector installed:
```bash
pip install snowflake-connector-python
```

In [ ]:
import snowflake.connector

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=snowflake_user,
    password=snowflake_password,
    account=snowflake_account,
    warehouse=snowflake_warehouse,
    database=snowflake_database,
    schema=snowflake_schema
)

cursor = conn.cursor()

In [ ]:
# Create database and schema if they don't exist
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {snowflake_database}")
cursor.execute(f"CREATE SCHEMA IF NOT EXISTS {snowflake_database}.{snowflake_schema}")
cursor.execute(f"USE SCHEMA {snowflake_database}.{snowflake_schema}")

# Create PaySim transactions table
create_table_sql = """
CREATE TABLE IF NOT EXISTS TRANSACTIONS (
   STEP INT,
   TYPE VARCHAR(50),
   AMOUNT FLOAT,
   NAMEORIG VARCHAR(100),
   OLDBALANCEORG FLOAT,
   NEWBALANCEORIG FLOAT,
   NAMEDEST VARCHAR(100),
   OLDBALANCEDEST FLOAT,
   NEWBALANCEDEST FLOAT,
   ISFRAUD INT,
   ISFLAGGEDFRAUD INT
   ) \
"""
cursor.execute(create_table_sql)

In [ ]:
# Download PaySim dataset
paysim_path = Path(kagglehub.dataset_download("ealaxi/paysim1"))
print("Path to paysim dataset files:", paysim_path)

# Upload CSV file to Snowflake stage and load into table
csv_file = next(paysim_path.glob('*.csv'))
cursor.execute("CREATE OR REPLACE STAGE paysim_stage")
cursor.execute(f"PUT file://{csv_file} @paysim_stage")
cursor.execute("""
COPY INTO TRANSACTIONS
FROM @paysim_stage
FILE_FORMAT = (TYPE = 'CSV' FIELD_DELIMITER = ',' SKIP_HEADER = 1)
ON_ERROR = 'CONTINUE'
""")

cursor.close()
conn.close()

In [ ]:
# Verify data loaded
cursor.execute("SELECT COUNT(*) FROM TRANSACTIONS")
count = cursor.fetchone()[0]
print(f"Found {count} transactions in Snowflake")

# close connection
cursor.close()
conn.close()

## Step 2: Set Up AWS Secrets Manager for Snowflake Credentials

Create a secret in AWS Secrets Manager to store Snowflake connection credentials.

**Create the secret using AWS CLI:**
```bash
aws secretsmanager create-secret \
    --name snowflake-credentials \
    --secret-string '{"username":"YOUR_SNOWFLAKE_USER","password":"YOUR_SNOWFLAKE_PASSWORD"}'
```

**Or via AWS Console:**
1. Go to AWS Secrets Manager → Secrets → Store a new secret
2. Select "Other type of secret"
3. Add key-value pairs:
   - Key: `username`, Value: Your Snowflake username
   - Key: `password`, Value: Your Snowflake password
4. Name: `snowflake-credentials`
5. Create secret and copy the ARN

In [ ]:
# Verify Secrets Manager secret exists
secrets_client = boto3.client('secretsmanager')

if snowflake_secret_arn:
    try:
        response = secrets_client.describe_secret(SecretId=snowflake_secret_arn)
        print(f"Secret found: {response['Name']}")
        print(f"ARN: {response['ARN']}")
    except secrets_client.exceptions.ResourceNotFoundException:
        print(f"ERROR: Secret not found: {snowflake_secret_arn}")
        print("Please create the secret as described above")
else:
    print("WARNING: SNOWFLAKE_SECRET_ARN not set")
    print("Set it using: %env SNOWFLAKE_SECRET_ARN=arn:aws:secretsmanager:region:account:secret:name")

## Step 3: Deploy Athena Snowflake Connector Lambda

Deploy the Athena Snowflake connector from AWS Serverless Application Repository.

**Manual Setup via AWS Console:**

1. Go to **AWS Serverless Application Repository**
2. Search for "AthenaSnowflakeConnector"
3. Click on the application and **Deploy**
4. Configure parameters:
   - **Application name**: `athena-snowflake-connector`
   - **SpillBucket**: S3 bucket for large query results (e.g., `my-athena-spill-bucket`)
   - **SecretNamePrefix**: `snowflake-credentials` (your secret name)
   - **SnowflakeConnectionString**: `snowflake://<account>.snowflakecomputing.com/?warehouse=<warehouse>&db=<database>&schema=<schema>`
5. Acknowledge IAM capabilities and **Deploy**
6. Wait for CloudFormation stack to complete (5-10 minutes)
7. Copy the Lambda function ARN from the stack outputs

**Alternative: Deploy via AWS CLI:**
```bash
# Create spill bucket if needed
aws s3 mb s3://my-athena-spill-bucket

# Deploy the connector
aws serverlessrepo create-cloud-formation-change-set \
    --application-id arn:aws:serverlessrepo:us-east-1:292517598671:applications/AthenaSnowflakeConnector \
    --stack-name athena-snowflake-connector \
    --capabilities CAPABILITY_IAM \
    --parameter-overrides \
        SpillBucket=my-athena-spill-bucket \
        SecretNamePrefix=snowflake-credentials \
        SnowflakeConnectionString="snowflake://<account>.snowflakecomputing.com/?warehouse=<warehouse>&db=<database>&schema=<schema>"
```

In [ ]:
# Verify Lambda function exists
lambda_client = boto3.client('lambda')
lambda_function_name = 'athena-snowflake-connector'

try:
    response = lambda_client.get_function(FunctionName=lambda_function_name)
    lambda_arn = response['Configuration']['FunctionArn']
    print(f"Lambda function found: {lambda_function_name}")
    print(f"ARN: {lambda_arn}")
except lambda_client.exceptions.ResourceNotFoundException:
    print(f"ERROR: Lambda function '{lambda_function_name}' not found")
    print("Please deploy the Athena Snowflake connector as described above")
    lambda_arn = None

## Step 4: Create Athena Data Catalog for Snowflake

Create an Athena data catalog that uses the Lambda connector to query Snowflake.

**Manual Setup via AWS Console:**

1. Go to **Athena Console** → **Data sources** → **Create data source**
2. Select **Query a data source** → **Snowflake**
3. Configure:
   - **Data source name**: `snowflake_catalog`
   - **Lambda function**: Select `athena-snowflake-connector`
4. Click **Create data source**

**Alternative: Create via AWS CLI:**
```bash
aws athena create-data-catalog \
    --name snowflake_catalog \
    --type LAMBDA \
    --parameters "function=arn:aws:lambda:REGION:ACCOUNT:function:athena-snowflake-connector"
```

In [ ]:
# Verify Athena catalog exists
athena_client = boto3.client('athena')

try:
    response = athena_client.get_data_catalog(Name=athena_catalog_name)
    print(f"Athena catalog '{athena_catalog_name}' found")
    print(f"Catalog type: {response['DataCatalog']['Type']}")
    if 'Parameters' in response['DataCatalog']:
        print(f"Lambda function: {response['DataCatalog']['Parameters'].get('function', 'N/A')}")
except athena_client.exceptions.InvalidRequestException:
    print(f"ERROR: Athena catalog '{athena_catalog_name}' not found")
    print("Please create the catalog manually as described above")

## Create Neptune Analytics Instance

Provision a Neptune Analytics instance or retrieve an existing one.

In [ ]:
session = SessionManager.session(session_name)
graph_list = session.list_graphs()
print("Available graphs:")
for g in graph_list:
    print(g)

In [ ]:
graph = await session.get_or_create_graph(config={"provisionedMemory": 32})
print(f"Retrieved graph: {graph}")

## Import Data from Snowflake via Athena Federated Query

Query Snowflake data through Athena using the Lambda connector and import into Neptune Analytics.

In [ ]:
# SQL queries to project nodes and edges from Snowflake
SOURCE_AND_DESTINATION_CUSTOMERS = f"""
SELECT DISTINCT "~id", 'customer' AS "~label"
FROM (
     SELECT NAMEORIG as "~id"
     FROM "{athena_catalog_name}"."{snowflake_database}"."{snowflake_schema}".TRANSACTIONS
     WHERE NAMEORIG IS NOT NULL
     UNION ALL
     SELECT NAMEDEST as "~id"
     FROM "{athena_catalog_name}"."{snowflake_database}"."{snowflake_schema}".TRANSACTIONS
     WHERE NAMEDEST IS NOT NULL
)
"""

BANK_TRANSACTIONS = f"""
SELECT
    NAMEORIG as "~from",
    NAMEDEST as "~to",
    TYPE AS "~label",
    STEP AS "step:Int",
    AMOUNT AS "amount:Float",
    OLDBALANCEORG AS "oldbalanceOrg:Float",
    NEWBALANCEORIG AS "newbalanceOrig:Float",
    OLDBALANCEDEST AS "oldbalanceDest:Float",
    NEWBALANCEDEST AS "newbalanceDest:Float",
    ISFRAUD AS "isFraud:Int",
    ISFLAGGEDFRAUD AS "isFlaggedFraud:Int"
FROM "{athena_catalog_name}"."{snowflake_database}"."{snowflake_schema}".TRANSACTIONS
WHERE NAMEORIG IS NOT NULL AND NAMEDEST IS NOT NULL
"""

await session.import_from_table(
    graph,
    s3_location_import,
    [SOURCE_AND_DESTINATION_CUSTOMERS, BANK_TRANSACTIONS],
    catalog=athena_catalog_name,
    database=snowflake_database
)

## Execute Louvain Algorithm for Fraud Detection

Run Louvain community detection to identify potential fraud networks.

In [ ]:
# Verify graph data
all_nodes = graph.execute_query("MATCH (n) RETURN n LIMIT 10")
print(f"Sample nodes: {all_nodes}")

all_edges = graph.execute_query("MATCH ()-[r]-() RETURN r LIMIT 10")
print(f"Sample edges: {all_edges}")

In [ ]:
# Run Louvain algorithm and mutate graph with community property
louvain_result = graph.execute_query(
    'CALL neptune.algo.louvain.mutate({iterationTolerance:1e-07, writeProperty:"community"}) '
    'YIELD success AS success RETURN success'
)
print(f"Louvain result: {louvain_result}")

## Export Enriched Graph Data to S3

Export the graph with community annotations back to S3 as Iceberg tables.

In [ ]:
csv_catalog = 'AwsDataCatalog'
csv_database = 'fraud_detection_results'
csv_table_name = 'transactions_csv'

iceberg_vertices_table_name = 'customers_with_communities'
iceberg_edges_table_name = 'transactions_enriched'
iceberg_catalog = 's3tablescatalog/fraud-detection-results'
iceberg_database = 'fraud_detection_results'

await session.export_to_table(
    graph["id"],
    s3_location_export,
    csv_table_name,
    csv_catalog,
    csv_database,
    iceberg_vertices_table_name,
    iceberg_edges_table_name,
    iceberg_catalog,
    iceberg_database
)

In [ ]:
# Clean up
session.destroy_all_graphs()

## Conclusion

This notebook demonstrated:

1. **Snowflake Setup**: Uploaded PaySim data to Snowflake and created a table
2. **AWS Secrets Manager**: Stored Snowflake credentials securely
3. **Athena Federated Query**: Deployed Lambda connector for Snowflake integration
4. **Data Import**: Queried Snowflake via Athena and imported into Neptune Analytics
5. **Graph Analytics**: Ran Louvain community detection for fraud pattern identification
6. **Data Export**: Exported enriched graph data back to S3 as Iceberg tables

This architecture enables graph analytics on Snowflake data without ETL pipelines, combining Snowflake's data warehouse capabilities with Neptune Analytics' graph algorithms through Athena Federated Query.

## Alternative: Using NetworkX API

You can also run algorithms using the NetworkX API with Neptune backend:

In [ ]:
import networkx as nx

nx.config.backends.neptune.graph_id = graph.graph_id

# Run Louvain with NetworkX API
result = nx.community.louvain_communities(
    nx.Graph(), 
    backend="neptune", 
    write_property="community"
)
print(f"Louvain result: {result}")